# 2D Fourier Transform

In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
images = [ cv2.imread("./data/00{}.jpg".format(i), cv2.IMREAD_GRAYSCALE) for i in range(1, 7)] 
import time

## Matrix Multiplication

In [5]:
class Matrix:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return self.data.__repr__()
    
    def __getitem__(self, i):
        return self.data[i]
    
    def __truediv__(self, denom):
        div_mat = self.zeros()
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                div_mat[i][j] = self[i][j] / denom 
        return Matrix(div_mat)
    
    def __len__(self):
        return len(self.data), len(self.data[0])
    @property 
    def zeros(self):
        return [[0 for _ in range(self.shape[1])] for _ in range(self.shape[0])]
    
    @property
    def shape(self):
        return (len(self.data), len(self.data[0]))
    
    def round(self, n=8):
        rounded = self.zeros
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                re = self.data[i][j].real
                im = self.data[i][j].imag
                if im == 0:
                    rounded[i][j] = round(re, n)
                else:
                    rounded[i][j] = round(re, n) + round(im, n) * 1j
        return Matrix(rounded)
        
    
    def matmul(self, B):
        # result = [[0 for _ in range(B.shape[1])] for _ in range(self.shape[0])]
        # for i in range(self.shape[0]):
        #     for j in range(B.shape[1]):
        #         sum = 0
        #         for k in range(self.shape[1]):
        #             sum += self[i][k] * B[k][j]
        #         result[i][j] = sum 
        # return Matrix(result)
        return Matrix(np.matmul(self.data, B.data))
    
    @property
    def transpose(self):
        result = [[0 for _ in range(self.shape[0])] for _ in range(self.shape[1])]
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[j][i] = self[i][j]
                
    @property
    def exp_mu(self, reverse = False):
        M = self.shape[0]
        result = [[0 for _ in range(M)] for _ in range(M)]
        for u in range(M):
            for m in range(M):
                if reverse:
                    result[m][u] = np.exp(2j* np.pi * m * u / M )
                else: 
                    result[m][u] = np.exp(-2j* np.pi * m * u / M )
        return Matrix(result)
    @property
    def exp_nu(self, reverse = True):
        N = self.shape[1]
        result = [[0 for _ in range(N)] for _ in range(N)]
        for v in range(N):
            for n in range(N):
                if reverse:
                    result[n][v] = np.exp(2j* np.pi * n * v / N )
                else:
                    result[n][v] = np.exp(-2j* np.pi * n * v / N )
                
        return Matrix(result)
    
    def re_im(self):
        re = self.zeros
        im = self.zeros
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                re[i][j] = self[i][j].real
                im[i][j] = self[i][j].imag
        return Matrix(re), Matrix(im) 
    
    @property
    def magnitude(self):
        mag = self.zeros
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                mag[i][j] = abs(self[i][j])
        return Matrix(mag)
    
    def shift(self):
        return Matrix(np.fft.fftshift(self.data))
    
    def show(self):
        mat = self.shift().magnitude.round(0)
        plt.imshow(mat.data, cmap='gray')
            
A = Matrix([[1, 2, 3], [4, 5, 6]])
B = Matrix([[1, 2], [3, 4], [5, 6]])

## 2D Fourier Transform

In [6]:
def fft2(img: Matrix): 
    M, N = img.shape 
    print("processing... might take a while")
    fourier = img.exp_mu.matmul(img).matmul(img.exp_nu) 
    fourier.round()
    print("process ended")
    return fourier

def inverse_fft2(img: Matrix):
    M, N = img.shape 
    inv_fourier = img.exp_nu(reversed = True).matmul(img).matmul(img.exp_mu(reversed = True))
    inv_fourier.round()
    return inv_fourier

In [7]:
img = Matrix(images[0])
fft_img = fft2(img)
print(fft_img.data)

processing... might take a while
process ended
[[ 6.45052100e+06     +0.j         -1.81076869e+06+358510.37551327j
  -3.63252791e+05+369521.00585044j ...  2.24452403e+05+134518.25566535j
  -3.63252791e+05-369521.00585043j -1.81076869e+06-358510.37551321j]
 [-1.12781460e+06-256690.12216659j  2.37613074e+05-174682.77095576j
   2.70990317e+05-408490.6023802j  ...  2.69694676e+04 +51928.24433348j
   4.74440241e+05+618894.58920607j  2.34476450e+05-460432.86490163j]
 [-1.17115301e+05-499075.19373302j  4.13450826e+05-298398.54516473j
  -1.84818004e+05-306781.99533241j ... -9.00537719e+04 -35861.77325114j
  -3.91914362e+03 +31039.63578145j  1.82997532e+05+106634.37018424j]
 ...
 [-6.09638567e+05+185655.85730444j  1.87700723e+05-426762.76471979j
   8.87406310e+04 +16969.25939901j ...  2.40760644e+05 -57806.53135033j
  -1.76640744e+05-368998.42652828j  2.84201452e+05+364306.50572387j]
 [-1.17115301e+05+499075.19373299j  1.82997532e+05-106634.37018419j
  -3.91914362e+03 -31039.63578148j ...  1.61

## Azimuthal Averaging

In [10]:
def azimuthal_averaging(img: Matrix):
    h, w = img.shape 
    center = (h // 2, w // 2) 
    max_radius = np.sqrt(center[0] ** 2 + center[1] ** 2) 
    if max_radius != int(max_radius):
        max_radius += 1
    max_radius = int(max_radius)
    cum_sum_freq = np.array([0 for _ in range(max_radius)])
    pixels = np.array([0 for _ in range(max_radius)])
    
    for i in range(h):
        for j in range(w):
            radius = np.sqrt((i - center[0]) ** 2 + (j - center[1]) ** 2)
            cum_sum_freq[int(radius)] += img[i][j]
            pixels[int(radius)] += 1
    cum_sum_freq = cum_sum_freq / pixels 
    cum_sum_freq = np.sort(cum_sum_freq)[::-1]
    return cum_sum_freq
    

In [11]:
fft_mag = fft_img.magnitude.round(0)
az = azimuthal_averaging(fft_mag)

In [13]:
az

array([3.38078567e+06, 5.43457500e+05, 4.04389875e+05, 2.08816067e+05,
       1.36831947e+05, 8.99875000e+04, 6.21369655e+04, 5.71527241e+04,
       4.02056667e+04, 3.25965814e+04, 2.60798537e+04, 2.14522264e+04,
       2.00154677e+04, 2.00085472e+04, 1.64366324e+04, 1.38988308e+04,
       1.33681467e+04, 1.18907237e+04, 1.05778276e+04, 9.37210753e+03,
       8.51364583e+03, 7.51977551e+03, 6.56214706e+03, 6.46656000e+03,
       5.16113274e+03, 5.00675781e+03, 4.78265217e+03, 4.69656204e+03,
       3.93227027e+03, 3.73497203e+03, 3.56433775e+03, 3.31323926e+03,
       2.98415294e+03, 2.78259195e+03, 2.70709091e+03, 2.57174603e+03,
       2.46885782e+03, 2.34934762e+03, 2.22760169e+03, 2.10112556e+03,
       2.04365203e+03, 1.98750279e+03, 1.57071080e+03, 1.49440647e+03,
       1.33817288e+03, 1.19954082e+03, 1.16588055e+03, 1.07526733e+03,
       1.03605517e+03, 9.94026756e+02, 9.66407801e+02, 9.20714286e+02,
       8.74769231e+02, 8.48269504e+02, 8.34533557e+02, 8.30622837e+02,
      